# train셋의 Bert_time을 이용하여 test셋에서 기후가 일치하는 Bert_time을 이용해 가능한 CI_HOUR를 계산

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
import bisect
%matplotlib inline

In [ ]:
train = pd.read_csv('./data/train.csv')

### small_train 만드는 과정

In [ ]:
small_train = train[['ARI_CO', 'ARI_PO', 'U_WIND', 'V_WIND', 'BN', 'AIR_TEMPERATURE','ATA','CI_HOUR']].copy()
# 결측치 제거
columns_to_check = ['U_WIND', 'V_WIND', 'BN', 'AIR_TEMPERATURE']
small_train = small_train.dropna(subset=columns_to_check)

# Bert_time 열 만들기 ('ATA' + 'CI_HOUR')
small_train['ATA'] = pd.to_datetime( small_train['ATA'])
small_train['CI_HOUR'] = pd.to_timedelta(small_train['CI_HOUR'], unit="hours")
small_train['Bert_time'] = small_train['ATA'] + small_train['CI_HOUR']
small_train.drop(columns='ATA', inplace=True)
small_train.drop(columns='CI_HOUR', inplace=True)
small_train

## 병렬 처리를 이용해 기후 6열이 일치하는 Bert_time을 구하여 CI_HOUR를 계산하고 리스트에 저장

In [4]:
test = pd.read_csv('./data/test.csv')
test['ATA'] = pd.to_datetime(test['ATA'])

# 일치하는 열을 사용하기 위해 필요한 컬럼 리스트 생성
columns_to_match = ['ARI_CO', 'ARI_PO', 'U_WIND', 'V_WIND', 'BN', 'AIR_TEMPERATURE']

In [5]:
def pad_float_fixed_length(x, total_length=10, decimals=6):
    format_str = "{:0" + str(total_length) + "." + str(decimals) + "f}"
    return format_str.format(x)

# 숫자를 일정한 길이의 문자열로 변환
for col in columns_to_match:
    if small_train[col].dtype in [np.float64, np.int64]:  # dtype을 numpy의 dtype으로 비교
        small_train[col] = small_train[col].apply(lambda x: pad_float_fixed_length(x))
    if test[col].dtype in [np.float64, np.int64]:
        test[col] = test[col].apply(lambda x: pad_float_fixed_length(x))

small_train = small_train.sort_values(by=columns_to_match).reset_index(drop=True)

train_key = small_train[columns_to_match].agg(''.join, axis=1)
test_key = test[columns_to_match].agg(''.join, axis=1)

In [6]:
def process_chunk(args):
    test_chunk_key, test_chunk_ata = args
    ci_hour_values_chunk = []

    for key, ata in zip(test_chunk_key, test_chunk_ata):
        index = bisect.bisect_left(train_key, key)
        matches = []

        while 0 <= index < len(train) and train_key.iloc[index] == key:
            matches.append(small_train.iloc[index])
            index += 1

        if matches:
            hours_deltas = [(row['Bert_time'] - ata) / pd.Timedelta('1 hour') for row in matches]
            filtered_deltas = [delta for delta in hours_deltas if delta >= 0]  # 0 이상인 값만 추가
            ci_hour_values_chunk.append(filtered_deltas if filtered_deltas else [-1])
        else:
            ci_hour_values_chunk.append([-1])

    return ci_hour_values_chunk

In [7]:
# CPU의 코어 개수를 얻기
num_cores = cpu_count()

# 데이터를 코어 개수만큼 분할
test_chunks_keys = np.array_split(test_key, num_cores)
test_chunks_ata = np.array_split(test['ATA'], num_cores)

# multiprocessing Pool을 사용하여 병렬 처리를 시작
with Pool(num_cores) as pool:
    results = pool.map(process_chunk, zip(test_chunks_keys, test_chunks_ata))

In [8]:
ci_hour_values = [item for sublist in results for item in sublist]

In [9]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['CI_HOUR'] = ci_hour_values
submit.to_csv('./out_csvs/use_bertTime_and_get_CI_HOURS.csv', index=False)